In [1]:
import cv2
import numpy as np

def ab(img):
    image = cv2.imread(img, cv2.IMREAD_GRAYSCALE)
    kernel = np.ones((5,5), np.uint8)
    closed_image = cv2.morphologyEx(image, cv2.MORPH_CLOSE, kernel)
    eroded_image = cv2.erode(closed_image, kernel, iterations=1)
    boundary_image = cv2.absdiff(closed_image, eroded_image)
    cv2.imshow('Original Image', image)
    cv2.imshow('Boundary Image', boundary_image)
    cv2.waitKey(0)
    cv2.destroyAllWindows()

    return boundary_image

boundary_image = ab('images.jpg')


In [3]:
import cv2
import numpy as np

def myMorphology(image_path):
    image = cv2.imread(image_path, cv2.IMREAD_GRAYSCALE)
    
    
    horizontal_kernel = cv2.getStructuringElement(cv2.MORPH_RECT, (9, 3))
    vertical_kernel = cv2.getStructuringElement(cv2.MORPH_RECT, (3, 9))
    horizontal_opening = cv2.morphologyEx(image, cv2.MORPH_OPEN, horizontal_kernel)
    vertical_opening = cv2.morphologyEx(image, cv2.MORPH_OPEN, vertical_kernel)

    horizontal_lines = cv2.bitwise_not(horizontal_opening)
    vertical_lines = cv2.bitwise_not(vertical_opening)

    cv2.imshow('Original Image', image)
    cv2.imshow('Horizontal Lines', horizontal_lines)
    cv2.imshow('Vertical Lines', vertical_lines)
    cv2.waitKey(0)
    cv2.destroyAllWindows()

myMorphology('a.jpg')


In [7]:
import cv2
import numpy as np

image = cv2.imread("a.jpg", cv2.IMREAD_GRAYSCALE)

binary_image = np.zeros_like(image)
threshold_value = 128 
for i in range(image.shape[0]):
    for j in range(image.shape[1]):
        binary_image[i, j] = 255 if image[i, j] < threshold_value else 0

kernel_size = 5
kernel = np.ones((kernel_size, kernel_size), dtype=np.uint8)
dilated_image = np.zeros_like(binary_image)

for i in range(kernel_size // 2, binary_image.shape[0] - kernel_size // 2):
    for j in range(kernel_size // 2, binary_image.shape[1] - kernel_size // 2):
        region = binary_image[
            i - kernel_size // 2 : i + kernel_size // 2 + 1,
            j - kernel_size // 2 : j + kernel_size // 2 + 1,
        ]
        if np.sum(region & kernel) > 0:  # Logical AND with the kernel
            dilated_image[i, j] = 255

visited = np.zeros_like(dilated_image, dtype=bool)
bounding_boxes = []

def flood_fill(image, x, y):
    """Flood fill to find a connected component."""
    stack = [(x, y)]
    min_x, min_y = x, y
    max_x, max_y = x, y
    while stack:
        cx, cy = stack.pop()
        if (
            0 <= cx < image.shape[0]
            and 0 <= cy < image.shape[1]
            and not visited[cx, cy]
            and image[cx, cy] == 255
        ):
            visited[cx, cy] = True
            min_x, min_y = min(min_x, cx), min(min_y, cy)
            max_x, max_y = max(max_x, cx), max(max_y, cy)
            stack.extend([(cx + dx, cy + dy) for dx, dy in [(-1, 0), (1, 0), (0, -1), (0, 1)]])
    return min_x, min_y, max_x, max_y

for i in range(dilated_image.shape[0]):
    for j in range(dilated_image.shape[1]):
        if dilated_image[i, j] == 255 and not visited[i, j]:
            bounding_boxes.append(flood_fill(dilated_image, i, j))

output_image = cv2.cvtColor(image, cv2.COLOR_GRAY2BGR)
for box in bounding_boxes:
    min_x, min_y, max_x, max_y = box
    cv2.rectangle(output_image, (min_y, min_x), (max_y, max_x), (0, 255, 0), 2)

cv2.imshow("Bounding Boxes", output_image)
cv2.imwrite("output_manual_bounding_boxes.png", output_image)
cv2.waitKey(0)
cv2.destroyAllWindows()
